In [1]:
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109
%load_ext autoreload
%autoreload 2
from AP_class import alch_deriv,first_deriv_nuc_nuc,second_deriv_nuc_nuc
from AP_class import APDFT_perturbator as AP
from AP_symmetry import benz_Symm

In [2]:
benz_atom=""" 
C         6.95191249e-13  2.61173060e+00 -1.67580194e-08 ;
H         2.83052087e-13  4.63941350e+00 -1.15976219e-08 ;
C         2.26182505e+00  1.30586530e+00 -2.37508246e-08 ; 
H         4.01784995e+00  2.31970675e+00 -9.84687205e-09 ; 
C         2.26182505e+00 -1.30586530e+00  8.26065094e-09 ; 
H         4.01784995e+00 -2.31970675e+00  5.83890508e-08 ; 
C        -8.45954456e-13 -2.61173060e+00 -1.55922267e-08 ; 
H         1.49445304e-12 -4.63941350e+00 -3.26774154e-08 ; 
C        -2.26182505e+00 -1.30586530e+00 -8.28266973e-09 ; 
H        -4.01784995e+00 -2.31970675e+00  1.87236306e-09 ; 
C        -2.26182505e+00  1.30586530e+00  1.68546675e-08 ; 
H        -4.01784995e+00  2.31970675e+00  3.31289175e-08 ; 
"""
Benz = gto.M(atom=benz_atom, basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])},unit='bohr')

In [3]:
benz_dm0=np.load("benz_dm.npy")
BenzHF=scf.RHF(Benz)
BenzHF.scf(dm0=benz_dm0)

converged SCF energy = -230.788695455662


-230.7886954556618

In [4]:
apbz=AP(BenzHF,sites=[0,2,4,6,8,10],symm=benz_Symm(Benz))

In [5]:
apbz.build_all()

In [6]:
apbz.APDFT3([1,2,0,0,0,1])-apbz.APDFT3([2,1,1,0,0,0])

1.0248868420603685e-10

In [7]:
apbz.af(0)

No AF found for atom 0. Calculating it from code


array([[-4.01172460e-13,  2.15252224e-02, -8.55259302e-10],
       [-4.24207902e-14,  7.83866964e-02,  6.30010539e-11],
       [ 6.76028944e-02, -3.29035758e-02, -9.91005130e-10],
       [ 9.50152738e-03, -5.77177984e-03,  3.70380648e-10],
       [ 2.91178600e-03,  9.40043921e-04, -1.87862576e-10],
       [ 1.70413431e-03, -5.34737950e-03,  2.06309676e-10],
       [ 1.28098468e-14, -1.13075776e-02,  1.81689004e-10],
       [ 5.88496131e-14, -2.43895882e-03,  2.29305228e-11],
       [-2.91178600e-03,  9.40043921e-04,  5.60654075e-12],
       [-1.70413431e-03, -5.34737950e-03, -9.19167489e-11],
       [-6.76028944e-02, -3.29035758e-02,  1.20204543e-09],
       [-9.50152738e-03, -5.77177984e-03,  7.40808812e-11]])

In [8]:
apbz.af(2)

array([[-6.22967797e-02,  4.20940361e-02,  1.20204543e-09],
       [-9.74927165e-03,  5.34267417e-03,  7.40808812e-11],
       [ 1.86413894e-02,  1.07626112e-02, -8.55259302e-10],
       [ 6.78848704e-02,  3.91933482e-02,  6.30010539e-11],
       [ 5.30611472e-03, -7.49976118e-02, -9.91005130e-10],
       [-2.47744274e-04, -1.11144540e-02,  3.70380648e-10],
       [ 2.26999491e-03, -2.05165868e-03, -1.87862576e-10],
       [-3.77889933e-03, -4.14951335e-03,  2.06309676e-10],
       [-9.79264947e-03, -5.65378881e-03,  1.81689004e-10],
       [-2.11220030e-03, -1.21947941e-03,  2.29305228e-11],
       [-6.41791082e-04,  2.99170260e-03,  5.60654075e-12],
       [-5.48303364e-03, -1.19786615e-03, -9.19167489e-11]])

In [9]:
from berny import Berny, geomlib, coords,Geometry

In [10]:
mut_list2=["NBNBCC","NBBNCC","BNNBCC","BBNNCC","NBNCBC","BNBCNC","BNNCBC","NBBCNC"]#4-11
mut_list3=["NBCBNC","NNCBBC","NBCNBC","NNNBBB","NBNBNB","NNBBNB"] #12-17

In [11]:
charge2symbol={1:"H",2:"He",3:"Li",4:"Be",5:"B",6:"C",7:"N",8:"O",9:"F",10:"Ne"}
symbol2charge={"H":1,"He":2,"Li":3,"Be":4,"B":5,"C":6,"N":7,"O":8,"F":9,"Ne":10}
af_array=np.asarray([apbz.af(0),apbz.af(2),apbz.af(4),apbz.af(6),apbz.af(8),apbz.af(10)])

In [12]:
np.einsum("ijk,i->jk",af_array,[1,0,1,0,0,0])-apbz.af(0)-apbz.af(4)

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -6.07153217e-18,  0.00000000e+00],
       [-3.46944695e-18,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  6.93889390e-18,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.08420217e-19,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  4.33680869e-19,  0.00000000e+00],
       [ 0.00000000e+00,  8.67361738e-19, -1.03397577e-25],
       [ 4.33680869e-19,  4.33680869e-19,  6.46234854e-27]])

In [13]:
[symbol2charge[x]-6 for x in mut_list2[1]]

[1, -1, -1, 1, 0, 0]

In [14]:
sum(apbz.af(2)-apbz.af(2))

array([0., 0., 0.])

In [15]:
for idx,mut in enumerate(mut_list2):
    targmol_refgeom=apbz.targ_mol(np.asarray([apbz.sites,[symbol2charge[x]-6 for x in mut]]))
    pbg=Geometry([charge2symbol[x] for x in targmol_refgeom.atom_charges()],targmol_refgeom.atom_coords()*bohr2ang)
    pbg.dump(open("./pred_geoms/{}_bg.xyz".format(idx+4),"w"),"xyz")
    bernyobj=Berny(pbg)
    bernyobj.send((apbz.APDFT3([symbol2charge[x]-6 for x in mut]),\
                   np.einsum("ijk,i->jk",af_array,[symbol2charge[x]-6 for x in mut]))) 
    bernyobj._state.geom.dump(open("./pred_geoms/{}_p.xyz".format(idx+4),"w"),"xyz")


In [16]:
for idx,mut in enumerate(mut_list3):
    targmol_refgeom=apbz.targ_mol(np.asarray([apbz.sites,[symbol2charge[x]-6 for x in mut]]))
    pbg=Geometry([charge2symbol[x] for x in targmol_refgeom.atom_charges()],targmol_refgeom.atom_coords()*bohr2ang)
    pbg.dump(open("./pred_geoms/{}_bg.xyz".format(idx+12),"w"),"xyz")
    bernyobj=Berny(pbg)
    bernyobj.send((apbz.APDFT3([symbol2charge[x]-6 for x in mut]),\
                   np.einsum("ijk,i->jk",af_array,[symbol2charge[x]-6 for x in mut]))) 
    bernyobj._state.geom.dump(open("./pred_geoms/{}_p.xyz".format(idx+12),"w"),"xyz")


In [40]:
import rmsd 

In [41]:
rmsd.main(args=["./pred_geoms/4_bg.xyz","./pred_geoms/4_p.xyz"])

0.062475342481590444
